<br>

# AMN




No que tange aos dados espaciais, a AMN mantem o portal [SIGMINE](https://geo.anm.gov.br/portal/apps/webappviewer/index.html?id=6a8f5ccc4b6a4c2bba79759aa952d908)



In [11]:
import tempfile
from pathlib import Path

import folium
import geopandas as gpd
from folium import plugins

import open_geodata as geo

<br>

---

## Lista Layers


In [ ]:
# Instancia classe IBGE
amn = geo.br.amn.ANM()

# Obtem Malhas de 2018
layers = amn.layers

# Resultados
layers.info()
layers.head()

<br>

---

## Obtem Dados


In [ ]:
gdf = amn.get_layers(layer='Processos Minerários Ativos - SP')
gdf.info()
gdf.head()

<br>

---

## Filtra Dados


In [ ]:
# Search
word_search = 'Mineração Righi Ltda Me'
word_search_outfilename = word_search.lower().replace(' ', '_')

# Select
gdf_interess = gdf.loc[gdf['NOME'].str.contains(word_search, case=False)]
gdf_interess

<br>

---

## Folium


In [ ]:
# PopUp
def popup_html(row):
    # Data
    # r01 = row['PROCESSO']
    # r02 = row['NUMERO']
    # r03 = row['ANO']
    r04 = row['AREA_HA']
    # r05 = row['ID']
    r06 = row['FASE']
    r07 = row['ULT_EVENTO']
    r08 = row['NOME']
    r09 = row['SUBS']
    # r10 = row['USO']
    # r11 = row['UF']
    r12 = row['DSProcesso']
    # r13 = row['geometry']

    # Infos
    return f"""
    <div>
    <h5>Processo {r12}</h5>
    <br><b>Nome:</b> {r08}
    <br><b>Substância:</b> {r09}
    <br>--------------------
    <br><b>Área (ha):</b> {r04}
    <br><b>Fase:</b> {r06}
    </div>
    """

<br>

---

### Create Layer


In [ ]:
def add_lyr_righi(geodataframe):
    # Input

    gdf = geodataframe.to_crs(epsg=4326)

    # Popup
    gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Layer
    lyr = folium.GeoJson(
        gdf,
        name='Processos Minerários - Righi',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        marker=folium.Marker(
            icon=folium.Icon(
                color='lightgray',
                icon_color='#FFFF00',
                # icon='leaf',
            ),
        ),
        tooltip=folium.GeoJsonTooltip(
            fields=['NOME'],
            aliases=['NOME'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

In [ ]:
def get_map(input_geojson):
    # Input
    # gdf = gpd.read_file(input_geojson)
    gdf = input_geojson.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]

    # Zoom
    min_zoom = 6
    max_zoom = 21
    padding = 1

    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0] * ((100 + padding) / 100),
        min_lon=bounds[0][1] * ((100 + padding) / 100),
        max_lat=bounds[1][0] * ((100 - padding) / 100),
        max_lon=bounds[1][1] * ((100 - padding) / 100),
        tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )

    # Add Layers
    m.add_child(geo.lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(geo.lyr.base.google_satellite(min_zoom, max_zoom))
    m.add_child(geo.lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(geo.lyr.base.google_streets(min_zoom, max_zoom))

    # Monitoramento
    m.add_child(add_lyr_righi(input_geojson))

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
with tempfile.TemporaryDirectory() as temp_dir:
    # Path
    temp_dir = Path(temp_dir)

    # Create Maps
    m = get_map(input_geojson=gdf_interess)

    # Save
    m.save(temp_dir / f'map_{word_search_outfilename}.html')

In [ ]:
m